<a href="https://colab.research.google.com/github/tripptytrip/Symbolic-Transformers/blob/main/train_symbolic_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Symbolic Transformer Training

Train a tiny transformer to predict next symbols in First-Order Logic formulas.

**What this does:**
- Generates synthetic FOL training data
- Trains a small transformer model (566K - 14M parameters)
- Learns syntax rules like: `∀` → must be followed by `VAR`

**Quick Start:** Run cells 1-4 in order. Training takes ~3 min/epoch on GPU.

---

## 1️⃣ Setup Environment
Clone the repository and install dependencies.

In [1]:
#@title 1. Setup Environment { display-mode: "form" }
#@markdown Run this cell first to set up the environment.

import os

# Check GPU
print("🔍 Checking GPU availability...")
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || print("⚠️ No GPU detected - training will be slower")

# Clone repository
print("\n📦 Cloning Symbolic-Transformers repository...")
%cd /content
if not os.path.exists('Symbolic-Transformers'):
    !git clone -q https://github.com/tripptytrip/Symbolic-Transformers.git
    print("✓ Repository cloned")
else:
    !cd Symbolic-Transformers && git pull -q
    print("✓ Repository updated")

%cd /content/Symbolic-Transformers

# Install dependencies
print("\n📚 Installing dependencies...")
!pip -q install numpy scipy pandas tqdm rich tensorboard
print("✓ Dependencies installed")

# Verify vocabulary
print("\n🔤 Verifying vocabulary...")
!python -c "from utils.vocabulary import Vocabulary; v = Vocabulary('unified_vocabulary.json'); print(f'✓ Vocabulary loaded: {v.vocab_size} tokens')"

print("\n" + "="*50)
print("✅ Setup complete! Proceed to Step 2.")
print("="*50)

🔍 Checking GPU availability...
/bin/bash: -c: line 1: syntax error near unexpected token `"⚠️ No GPU detected - training will be slower"'
/bin/bash: -c: line 1: `nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || print("⚠️ No GPU detected - training will be slower")'

📦 Cloning Symbolic-Transformers repository...
/content
✓ Repository cloned
/content/Symbolic-Transformers

📚 Installing dependencies...
✓ Dependencies installed

🔤 Verifying vocabulary...
✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']
✓ Vocabulary loaded: 662 tokens

✅ Setup complete! Proceed to Step 2.


## 2️⃣ Configure Training
Adjust the settings below, then run the cell to apply them.

In [2]:
#@title 2. Training Configuration { display-mode: "form" }

#@markdown ### 🏗️ Model Size
model_size = "base" #@param ["tiny", "small", "base"]
#@markdown - **tiny**: 566K params (~2.2MB) - Fast training, good for experiments
#@markdown - **small**: 3.5M params (~14MB) - Better accuracy, moderate training time
#@markdown - **base**: 14M params (~56MB) - Best accuracy, longest training

#@markdown ---
#@markdown ### 📊 Dataset Size
num_train_formulas = 10000 #@param {type:"slider", min:100, max:10000, step:100}
#@markdown Number of unique FOL formulas to generate for training.
#@markdown - 100-500: Quick test runs
#@markdown - 1000-3000: Standard training
#@markdown - 5000-10000: Large-scale training (recommended for best results)

num_val_formulas = 1000 #@param {type:"slider", min:50, max:1000, step:50}
#@markdown Number of formulas for validation (typically 10-20% of training).

#@markdown ---
#@markdown ### ⚙️ Training Parameters
num_epochs = 200 #@param {type:"slider", min:10, max:1000, step:10}
#@markdown Number of training epochs.
#@markdown - 10-50: Quick experiments
#@markdown - 100-300: Standard training
#@markdown - 500-1000: Train to convergence (watch for overfitting!)

#@markdown ---
#@markdown ### 💾 Resume from Checkpoint
resume_training = False #@param {type:"boolean"}
#@markdown Resume from the last saved checkpoint.

# Store configuration
config = {
    'model_size': model_size,
    'num_train_formulas': num_train_formulas,
    'num_val_formulas': num_val_formulas,
    'num_test_formulas': max(50, num_val_formulas // 2),
    'num_epochs': num_epochs,
    'resume': resume_training
}

# Display configuration summary
print("="*50)
print("📋 TRAINING CONFIGURATION")
print("="*50)

model_params = {'tiny': '566K', 'small': '3.5M', 'base': '14M'}
model_size_mb = {'tiny': '~2.2MB', 'small': '~14MB', 'base': '~56MB'}

print(f"\n🏗️  Model:     {model_size} ({model_params[model_size]} parameters, {model_size_mb[model_size]})")
print(f"📊 Dataset:   {num_train_formulas} train / {num_val_formulas} val formulas")
print(f"⚙️  Training:  {num_epochs} epochs")
print(f"🔄 Resume:    {'Yes' if resume_training else 'No (fresh start)'}")

# Estimate training time
samples_per_formula = 14  # Average tokens per formula
total_samples = num_train_formulas * samples_per_formula
batch_size = 64  # Default in train.py
batches_per_epoch = total_samples // batch_size
time_per_batch = {'tiny': 0.012, 'small': 0.025, 'base': 0.05}  # seconds on T4
est_epoch_time = batches_per_epoch * time_per_batch[model_size]
est_total_time = est_epoch_time * num_epochs / 60

print(f"\n⏱️  Estimated training time: ~{est_total_time:.1f} minutes on GPU")
print(f"   ({est_epoch_time:.1f}s per epoch, {batches_per_epoch} batches)")

# Overfitting warning
if num_epochs > 200 and num_train_formulas < 3000:
    print(f"\n⚠️  WARNING: High epochs ({num_epochs}) with small dataset ({num_train_formulas})")
    print(f"   Risk of overfitting! Consider:")
    print(f"   - Reducing epochs to 100-200")
    print(f"   - Or increasing training formulas to 5000+")

print("\n" + "="*50)
print("✅ Configuration saved! Proceed to Step 3.")
print("="*50)

📋 TRAINING CONFIGURATION

🏗️  Model:     base (14M parameters, ~56MB)
📊 Dataset:   10000 train / 1000 val formulas
⚙️  Training:  200 epochs
🔄 Resume:    No (fresh start)

⏱️  Estimated training time: ~364.5 minutes on GPU
   (109.4s per epoch, 2187 batches)

✅ Configuration saved! Proceed to Step 3.


## 3️⃣ Generate Training Data
Create synthetic First-Order Logic formulas for training.

In [3]:
#@title 3. Generate Training Data { display-mode: "form" }
#@markdown This generates synthetic FOL formulas like:
#@markdown - `∀x₁ (P₅(x₁) → Q₂(x₁))`
#@markdown - `∃x₀ ∃x₁ (R₃(x₀, x₁) ∧ P₁(x₀))`

print("🔄 Generating training data...")
print(f"   Train: {config['num_train_formulas']} formulas")
print(f"   Val:   {config['num_val_formulas']} formulas")
print(f"   Test:  {config['num_test_formulas']} formulas")
print()

# Generate data using the dataset generator
import sys
sys.path.insert(0, '/content/Symbolic-Transformers')

from data.dataset_generator import generate_training_data

generate_training_data(
    vocab_path="unified_vocabulary.json",
    output_dir="datasets/fol_next_symbol",
    n_train=config['num_train_formulas'],
    n_val=config['num_val_formulas'],
    n_test=config['num_test_formulas'],
)

# Show dataset stats
print("\n📁 Dataset files:")
!ls -lh datasets/fol_next_symbol/*.json

print("\n" + "="*50)
print("✅ Data generated! Proceed to Step 4.")
print("="*50)

🔄 Generating training data...
   Train: 10000 formulas
   Val:   1000 formulas
   Test:  500 formulas

FOL DATASET GENERATION
✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']

Generating train set (10000 formulas)...
  Complexity 1: 2000 formulas
  Complexity 2: 4000 formulas
  Complexity 3: 3000 formulas
  Complexity 4: 1000 formulas
✓ Saved 144686 samples to datasets/fol_next_symbol/train.json

Generating val set (1000 formulas)...
  Complexity 1: 200 formulas
  Complexity 2: 400 formulas
  Complexity 3: 300 formulas
  Complexity 4: 100 formulas
✓ Saved 14614 samples to datasets/fol_next_symbol/val.json

Generating test set (500 formulas)...
  Complexity 1: 100 formulas
  Complexity 2: 200 formulas
  Complexity 3: 150 formulas
  Complexity 4: 50 formulas
✓ Saved 6895 samples to datasets/fol_next_symbol/test.json

✓ Dataset generation complete!
✓ Output directory: datasets/fol_next_symbol
✓ Train sampl

## 4️⃣ Train the Model
Start training! Watch the loss decrease over epochs.

In [ ]:
#@title 4. Train Model { display-mode: "form" }
#@markdown Training will begin with the configuration from Step 2.
#@markdown
#@markdown **What to watch for:**
#@markdown - Loss should decrease over epochs
#@markdown - Val loss < 1.0 is good progress
#@markdown - Val loss < 0.9 is excellent
#@markdown - `[BEST]` indicates a new best checkpoint was saved
#@markdown - **Stop if val loss starts rising** (overfitting)

import os
os.chdir('/content/Symbolic-Transformers')

print("="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"Model: {config['model_size']} | Epochs: {config['num_epochs']}")
if config['resume']:
    print("Resuming from last checkpoint...")
print("="*60 + "\n")

# Build training command with only supported arguments
cmd = f"python training/train.py --model-size {config['model_size']} --num-epochs {config['num_epochs']}"
if config['resume']:
    cmd += " --resume"

# Run training
!{cmd}

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print("\n📁 Saved checkpoints:")
!ls -lh checkpoints/*.pt 2>/dev/null | tail -5
print("\nBest model saved to: checkpoints/best_model.pt")

🚀 STARTING TRAINING
Model: base | Epochs: 200

✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']

✓ Loaded vocabulary: 662 tokens

Loading datasets...
✓ Loaded 144686 samples from datasets/fol_next_symbol/train.json
✓ Loaded 14614 samples from datasets/fol_next_symbol/val.json
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
✓ Train batches: 9043
✓ Val batches: 914

Creating model...
✓ Created base model with 19,593,878 parameters
✓ Using device: cuda
✓ GPU: Tesla T4
✓ VRAM: 15.8 GB

TRAINING START
Model: 

## 5️⃣ Evaluate Model (Optional)
Run evaluation on the test set to see accuracy metrics.

In [ ]:
#@title 5. Evaluate Model { display-mode: "form" }
#@markdown Run evaluation on the test set.

import os
os.chdir('/content/Symbolic-Transformers')

print("📊 Evaluating model on test set...\n")

!python evaluate_model.py \
    --checkpoint checkpoints/best_model.pt \
    --test-data datasets/fol_next_symbol/test.json

## 6️⃣ Download Trained Model (Optional)
Download the trained model checkpoint to your local machine.

In [ ]:
#@title 6. Download Model { display-mode: "form" }
#@markdown Download the best model checkpoint.

from google.colab import files
import os

checkpoint_path = '/content/Symbolic-Transformers/checkpoints/best_model.pt'

if os.path.exists(checkpoint_path):
    print(f"📦 Preparing download...")
    file_size = os.path.getsize(checkpoint_path) / (1024 * 1024)
    print(f"   File: best_model.pt ({file_size:.1f} MB)")
    print(f"   Model: {config['model_size']}")
    print()
    files.download(checkpoint_path)
else:
    print("❌ No checkpoint found. Run training first (Step 4).")

## 7️⃣ Interactive Demo (Optional)
Try the model interactively - type tokens and see predictions!

In [ ]:
#@title 7. Quick Demo { display-mode: "form" }
#@markdown See the model's predictions for a sample input.

import torch
import sys
sys.path.insert(0, '/content/Symbolic-Transformers')

from utils.vocabulary import Vocabulary
from models.transformer import SymbolicTransformer, get_model_config

# Load model
print("🔄 Loading model...")
vocab = Vocabulary('unified_vocabulary.json')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint = torch.load('checkpoints/best_model.pt', map_location=device, weights_only=False)
model_config = get_model_config(checkpoint['config']['model_size'], vocab.vocab_size)
model = SymbolicTransformer(**model_config).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✓ Loaded {checkpoint['config']['model_size']} model from epoch {checkpoint['epoch']}")
print(f"  Val loss: {checkpoint['best_val_loss']:.4f}\n")

# Demo predictions
def predict_next(tokens, top_k=5):
    """Predict next token given a sequence."""
    token_ids = [vocab.encode_label(t) if t in vocab.label_to_id else int(t) for t in tokens]
    x = torch.tensor([token_ids], device=device)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits[0, -1], dim=-1)
        top_probs, top_ids = torch.topk(probs, top_k)

    print(f"Input: {' '.join(tokens)}")
    print(f"\nTop {top_k} predictions:")
    for i, (prob, tid) in enumerate(zip(top_probs, top_ids)):
        label = vocab.decode_id(tid.item())
        bar = '█' * int(prob * 30)
        print(f"  {i+1}. {label:12s} {prob*100:5.1f}% {bar}")
    print()

# Show example predictions
print("="*50)
print("📊 EXAMPLE PREDICTIONS")
print("="*50 + "\n")

# After FORALL, should predict VAR with high confidence
predict_next(['FORALL'])

# After FORALL VAR, should predict a numeral
predict_next(['FORALL', 'VAR'])

# After a complete variable binding
predict_next(['FORALL', 'VAR', '1'])

print("\n💡 The model learned that FORALL must be followed by VAR!")

---

## 📖 Quick Reference

### Model Sizes
| Size | Parameters | File Size | Training Speed | Accuracy |
|------|-----------|-----------|----------------|----------|
| tiny | 566K | ~2.2MB | ~3s/epoch | Good |
| small | 3.5M | ~14MB | ~8s/epoch | Better |
| base | 14M | ~56MB | ~20s/epoch | Best |

### Recommended Configurations

**Quick Test (5 min):**
- Model: tiny
- Data: 500 formulas
- Epochs: 20

**Standard Training (30 min):**
- Model: tiny
- Data: 1000 formulas  
- Epochs: 100

**Best Results (2+ hours):**
- Model: small
- Data: 5000+ formulas
- Epochs: 200-300 (watch val loss!)

### Interpreting Results
- **Val Loss > 2.0**: Model is still learning basic patterns
- **Val Loss 1.0-2.0**: Good progress, learning syntax rules
- **Val Loss < 1.0**: Excellent, model understands FOL structure
- **Val Loss < 0.9**: Very good, approaching capacity limits

### ⚠️ Overfitting Warning Signs
- Train loss keeps dropping but val loss stops improving
- Val loss starts **increasing** while train loss decreases
- Gap between train and val loss > 0.2

**If overfitting:** Stop training and use the checkpoint with lowest val loss.

### Files Created
- `checkpoints/best_model.pt` - Best performing model
- `checkpoints/checkpoint_epoch_N.pt` - Periodic checkpoints
- `datasets/fol_next_symbol/` - Training data